In [ ]:
# Modification of unreasonable DEWP SLP STP data in NORO

In [1]:
import pandas as pd
import numpy as np

In [2]:
noro = pd.read_csv('/Users/evelyn/Desktop/Noro.csv')

In [3]:
#download data
noro_station = pd.read_csv('/Users/evelyn/Desktop/weather information/noro_station.csv')

noro_station = noro_station.dropna(subset = ['USAF'])
noro_station = noro_station.reset_index(drop = True)

noro_station["WBAN"] = pd.to_numeric(noro_station["WBAN"], errors='coerce').fillna('0').astype('int32')
for a in range(len(noro_station)):
    wban_unmodified = noro_station.loc[a, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        noro_station.loc[a, "WBAN"] = str(wban_unmodified).zfill(5)


In [4]:
#download noro_null data
noro_null = pd.read_csv('/Users/evelyn/Desktop/weather information/noro_interpolated stations.csv')

for z in range(len(noro_null)):
    noro_null.loc[z, "USAF0"] = str(noro_null.loc[z, "USAF0"])
    noro_null.loc[z, "WBAN0"] = str(int(noro_null.loc[z, "WBAN0"]))

    if not(pd.isna(noro_null.loc[z, "USAF1"])):
        noro_null.loc[z, "USAF1"] = str(noro_null.loc[z, "USAF1"])
        noro_null.loc[z, "WBAN1"] = str(int(noro_null.loc[z, "WBAN1"]))
        
    if not(pd.isna(noro_null.loc[z, "USAF2"])):
        noro_null.loc[z, "USAF2"] = str(noro_null.loc[z, "USAF2"])
        noro_null.loc[z, "WBAN2"] = str(int(noro_null.loc[z, "WBAN2"]))

In [5]:
#recaculate the three interpolation data
import numpy as np
path = '/Users/evelyn/Desktop/project/noro_null/'
path0 = '/Users/evelyn/Desktop/weather information/noro_interpolated/'

store_path = '/Users/evelyn/Desktop/NORO/'
variable = ["TEMP", "DEWP", "SLP", "STP", "WDSP", "MXSPD", "MAX", "MIN","PRCP"]

for i in range(len(noro_null)):
    print('{:.4%}'.format(i/len(noro_null)))
    if(pd.isna(noro_null.loc[i, "USAF2"])):
        continue
    else:
        file = []
        startyear = str(noro_null.loc[i , "InitialExposure"])[:4]
        endyear = str(noro_null.loc[i , "LastExposure"])[:4]
        
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(3):
                usaf = noro_null.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(noro_null.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            dfw = pd.read_csv(path0 + file[0])
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            df2 = pd.read_csv(path + file[2])
            
            for n in range(len(dfw)):
                if(len(df0[df0.DATE.str.contains(dfw.loc[n, "DATE"])]) == 0 or len(df1[df1.DATE == dfw.loc[n, "DATE"]]) == 0 or len(df2[df2.DATE == dfw.loc[n, "DATE"]]) == 0):
                    dfw = dfw.drop(index = n)
                else:
                    index0 = df0[df0.DATE.str.contains(dfw.loc[n, "DATE"])].index.tolist()[0]
                    index1 = df1[df1.DATE == dfw.loc[n, "DATE"]].index.tolist()[0]
                    index2 = df2[df2.DATE == dfw.loc[n, "DATE"]].index.tolist()[0]
                    #对TEMP进行修改
                    TEMP = [float(df0.loc[index0, "TEMP"]), float(df1.loc[index1, "TEMP"]), float(df2.loc[index2, "TEMP"])]
                    wt = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for temp in range(3):
                        if(TEMP[temp] > 9999):
                            TEMP[temp] = 0
                            wt[temp] = 0
                    zt = sum(wt)
                    if(max(wt) > 0):
                        wt[0] = wt[0]/zt
                        wt[1] = wt[1]/zt
                        wt[2] = wt[2]/zt
                        dfw.loc[n, "TEMP"] = round(sum(np.multiply(TEMP,wt)), 2)
                    else:
                        dfw.loc[n, "TEMP"] = np.nan
                    #DEWP
                    DEWP = [float(df0.loc[index0, "DEWP"]), float(df1.loc[index1, "DEWP"]), float(df2.loc[index2, "DEWP"])]
                    wd = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for dewp in range(3):
                        if(DEWP[dewp] > 9999):
                            DEWP[dewp] = 0
                            wt[dewp] = 0
                    zd = sum(wd)
                    if(max(wd) > 0):
                        wd[0] = wd[0]/zd
                        wd[1] = wd[1]/zd
                        wd[2] = wd[2]/zd
                        dfw.loc[n, "DEWP"] = round(sum(np.multiply(DEWP,wd)), 2)
                    else:
                        dfw.loc[n, "DEWP"] = np.nan
                    #SLP
                    SLP = [float(df0.loc[index0, "SLP"]), float(df1.loc[index1, "SLP"]), float(df2.loc[index2, "SLP"])]
                    wslp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for slp in range(3):
                        if(SLP[slp] > 9999):
                            SLP[slp] = 0
                            wslp[slp] = 0
                    zslp = sum(wslp)
                    if(max(wslp) > 0):
                        wslp[0] = wslp[0]/zslp
                        wslp[1] = wslp[1]/zslp
                        wslp[2] = wslp[2]/zslp
                        dfw.loc[n, "SLP"] = round(sum(np.multiply(SLP,wslp)), 2)
                    else:
                        dfw.loc[n, "SLP"] = np.nan
                    #STP
                    STP = [float(df0.loc[index0, "STP"]), float(df1.loc[index1, "STP"]), float(df2.loc[index2, "STP"])]
                    STP_ATTRIBUTES = [float(df0.loc[index0, "STP_ATTRIBUTES"]), float(df1.loc[index1, "STP_ATTRIBUTES"]), float(df2.loc[index2, "STP_ATTRIBUTES"])]
                    wstp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for stp in range(3):
                        if(STP[stp] > 9999):
                            STP[stp] = 0
                            wstp[stp] = 0
                        elif(STP[stp] < 100):
                            STP[stp] = STP[stp] + 1000
                        elif(STP_ATTRIBUTES[stp] == 0):
                            STP[stp] = 0
                            wstp[stp] = 0
                    zstp = sum(wstp)
                    if(max(wstp) >0):
                        wstp[0] = wstp[0]/zstp
                        wstp[1] = wstp[1]/zstp
                        wstp[2] = wstp[2]/zstp
                        dfw.loc[n, "STP"] = round(sum(np.multiply(STP,wstp)), 2)
                    else:
                        dfw.loc[n, "STP"] = np.nan
                    #WDSP
                    WDSP = [float(df0.loc[index0, "WDSP"]), float(df1.loc[index1, "WDSP"]), float(df2.loc[index2, "WDSP"])]
                    wwdsp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for wdsp in range(3):
                        if(WDSP[wdsp] > 999):
                            WDSP[wdsp] = 0
                            wwdsp[wdsp] = 0
                    zwdsp = sum(wwdsp)
                    if(max(wwdsp) > 0):
                        wwdsp[0] = wwdsp[0]/zwdsp
                        wwdsp[1] = wwdsp[1]/zwdsp
                        wwdsp[2] = wwdsp[2]/zwdsp
                        dfw.loc[n, "WDSP"] = round(sum(np.multiply(WDSP,wwdsp)), 2)
                    else:
                        dfw.loc[n, "WDSP"] = np.nan
                    #MXSPD
                    MXSPD = [float(df0.loc[index0, "MXSPD"]), float(df1.loc[index1, "MXSPD"]), float(df2.loc[index2, "MXSPD"])]
                    wmx = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for mxspd in range(3):
                        if(MXSPD[mxspd] > 999):
                            MXSPD[mxspd] = 0
                            wmx[mxspd] = 0
                    zmx = sum(wmx)
                    if(max(wmx) > 0):
                        wmx[0] = wmx[0]/zmx
                        wmx[1] = wmx[1]/zmx
                        wmx[2] = wmx[2]/zmx
                        dfw.loc[n, "MXSPD"] = round(sum(np.multiply(MXSPD,wmx)), 2)
                    else:
                        dfw.loc[n, "MXSPD"] = np.nan
                    #MAX
                    MAX = [float(df0.loc[index0, "MAX"]), float(df1.loc[index1, "MAX"]), float(df2.loc[index2, "MAX"])]
                    wtm = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for tmax in range(3):
                        if(MAX[tmax] > 9999):
                            MAX[tmax] = 0
                            wtm[tmax] = 0
                    ztm = sum(wtm)
                    if(max(wtm) > 0):
                        wtm[0] = wtm[0]/ztm
                        wtm[1] = wtm[1]/ztm
                        wtm[2] = wtm[2]/ztm
                        dfw.loc[n, "MAX"] = round(sum(np.multiply(MAX,wtm)), 2)
                    else:
                        dfw.loc[n, "MAX"] = np.nan
                    #MIN
                    MIN = [float(df0.loc[index0, "MIN"]), float(df1.loc[index1, "MIN"]), float(df2.loc[index2, "MIN"])]
                    wtmi = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for tmin in range(3):
                        if(MIN[tmin] > 9999):
                            MIN[tmin] = 0
                            wtmi[tmin] = 0
                    ztmi = sum(wtmi)
                    if(max(wtmi)>0):
                        wtmi[0] = wtmi[0]/ztmi
                        wtmi[1] = wtmi[1]/ztmi
                        wtmi[2] = wtmi[2]/ztmi
                        dfw.loc[n, "MIN"] = round(sum(np.multiply(MIN,wtmi)), 2)
                    else:
                        dfw.loc[n, "MIN"] = np.nan
                    #PRCP
                    PRCP = [float(df0.loc[index0, "PRCP"]), float(df1.loc[index1, "PRCP"]), float(df2.loc[index2, "PRCP"])]
                    wp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"]), float(noro_null.loc[i, "W2"])]
                    for temp in range(3):
                        if(PRCP[temp] > 99):
                            PRCP[temp] = 0
                            wp[temp] = 0
                    zp = sum(wp)
                    if(max(wp) > 0):
                        wp[0] = wp[0]/zp
                        wp[1] = wp[1]/zp
                        wp[2] = wp[2]/zp
                        dfw.loc[n, "PRCP"] = round(sum(np.multiply(PRCP,wp)), 2)
                    else:
                        dfw.loc[n, "PRCP"] = np.nan
                
            new_usaf = noro_null.loc[i, "USAF0"]
            new_wban = pd.to_numeric(noro_null.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            dfw.to_csv(store_path + new_filename)
                    

0.0000%
0.3413%
0.6826%
1.0239%
1.3652%
1.7065%
2.0478%
2.3891%
2.7304%
3.0717%
3.4130%
3.7543%
4.0956%
4.4369%
4.7782%
5.1195%
5.4608%
5.8020%
6.1433%
6.4846%
6.8259%
7.1672%
7.5085%
7.8498%
8.1911%
8.5324%
8.8737%
9.2150%
9.5563%
9.8976%
10.2389%
10.5802%
10.9215%
11.2628%
11.6041%
11.9454%
12.2867%
12.6280%
12.9693%
13.3106%
13.6519%
13.9932%
14.3345%
14.6758%
15.0171%
15.3584%
15.6997%
16.0410%
16.3823%
16.7235%
17.0648%
17.4061%
17.7474%
18.0887%
18.4300%
18.7713%
19.1126%
19.4539%
19.7952%
20.1365%
20.4778%
20.8191%
21.1604%
21.5017%
21.8430%
22.1843%
22.5256%
22.8669%
23.2082%
23.5495%
23.8908%
24.2321%
24.5734%
24.9147%
25.2560%
25.5973%
25.9386%
26.2799%
26.6212%
26.9625%
27.3038%
27.6451%
27.9863%
28.3276%
28.6689%
29.0102%
29.3515%
29.6928%
30.0341%
30.3754%
30.7167%
31.0580%
31.3993%
31.7406%
32.0819%
32.4232%
32.7645%
33.1058%
33.4471%
33.7884%
34.1297%
34.4710%
34.8123%
35.1536%
35.4949%
35.8362%
36.1775%
36.5188%
36.8601%
37.2014%
37.5427%
37.8840%
38.2253%
38.5666%
38.9

In [6]:
#two interpolation data
import numpy as np
path = '/Users/evelyn/Desktop/project/noro_null/'

store_path = '/Users/evelyn/Desktop/NORO/'
variable = ["TEMP", "DEWP", "SLP", "STP", "WDSP", "MXSPD", "MAX", "MIN","PRCP"]

for i in range(len(noro_null)):
    print('{:.4%}'.format(i/len(noro_null)))
    if(pd.isna(noro_null.loc[i, "USAF2"])):
        file = []
        startyear = str(noro_null.loc[i , "InitialExposure"])[:4]
        endyear = str(noro_null.loc[i , "LastExposure"])[:4]
        
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(2):
                usaf = noro_null.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(noro_null.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            
            for j in range(len(df0)):
                date_string = df0.loc[j, "DATE"]
                if not(df1.DATE.str.contains(date_string).any()):
                    df0 = df0.drop(index = [j])
            df0 = df0.reset_index(drop = True)

            for n in range(len(df0)):

                index1 = df1[df1.DATE.str.contains(df0.loc[n, "DATE"])].index.tolist()[0]

                #对TEMP进行修改
                TEMP = [float(df0.loc[n, "TEMP"]), float(df1.loc[index1, "TEMP"])]
                wt = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for temp in range(2):
                    if(TEMP[temp] > 9999):
                        TEMP[temp] = 0
                        wt[temp] = 0
                zt = sum(wt)
                if(max(wt) > 0):
                    wt[0] = wt[0]/zt
                    wt[1] = wt[1]/zt
                    df0.loc[n, "TEMP"] = round(sum(np.multiply(TEMP,wt)), 2)
                else:
                    df0.loc[n, "TEMP"] = np.nan

                #DEWP
                DEWP = [float(df0.loc[n , "DEWP"]), float(df1.loc[index1, "DEWP"])]
                wd = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for dewp in range(2):
                    if(DEWP[dewp] > 9999):
                        DEWP[dewp] = 0
                        wt[dewp] = 0
                zd = sum(wd)
                if(max(wd) > 0):
                    wd[0] = wd[0]/zd
                    wd[1] = wd[1]/zd
                    df0.loc[n, "DEWP"] = round(sum(np.multiply(DEWP,wd)), 2)
                else:
                    df0.loc[n, "DEWP"] = np.nan
                #SLP
                SLP = [float(df0.loc[n, "SLP"]), float(df1.loc[index1, "SLP"])]
                wslp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for slp in range(2):
                    if(SLP[slp] > 9999):
                        SLP[slp] = 0
                        wslp[slp] = 0
                zslp = sum(wslp)
                if(max(wslp) > 0):
                    wslp[0] = wslp[0]/zslp
                    wslp[1] = wslp[1]/zslp
                    df0.loc[n, "SLP"] = round(sum(np.multiply(SLP,wslp)), 2)
                else:
                    df0.loc[n, "SLP"] = np.nan

                #STP
                STP = [float(df0.loc[n, "STP"]), float(df1.loc[index1, "STP"])]
                STP_ATTRIBUTES = [float(df0.loc[n, "STP_ATTRIBUTES"]), float(df1.loc[index1, "STP_ATTRIBUTES"])]
                wstp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for stp in range(2):
                    if(STP[stp] > 9999):
                        STP[stp] = 0
                        wstp[stp] = 0
                    elif(STP[stp] < 100):
                        STP[stp] = STP[stp] + 1000
                    elif(STP_ATTRIBUTES[stp] == 0):
                        STP[stp] = 0
                        wstp[stp] = 0
                zstp = sum(wstp)
                if(max(wstp) >0):
                    wstp[0] = wstp[0]/zstp
                    wstp[1] = wstp[1]/zstp
                    df0.loc[n, "STP"] = round(sum(np.multiply(STP,wstp)), 2)
                else:
                    df0.loc[n, "STP"] = np.nan
                    
                #WDSP
                WDSP = [float(df0.loc[n, "WDSP"]), float(df1.loc[index1, "WDSP"])]
                wwdsp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for wdsp in range(2):
                    if(WDSP[wdsp] > 999):
                        WDSP[wdsp] = 0
                        wwdsp[wdsp] = 0
                zwdsp = sum(wwdsp)
                if(max(wwdsp) > 0):
                    wwdsp[0] = wwdsp[0]/zwdsp
                    wwdsp[1] = wwdsp[1]/zwdsp
                    df0.loc[n, "WDSP"] = round(sum(np.multiply(WDSP,wwdsp)), 2)
                else:
                    df0.loc[n, "WDSP"] = np.nan
                #MXSPD
                MXSPD = [float(df0.loc[n, "MXSPD"]), float(df1.loc[index1, "MXSPD"])]
                wmx = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for mxspd in range(2):
                    if(MXSPD[mxspd] > 999):
                        MXSPD[mxspd] = 0
                        wmx[mxspd] = 0
                zmx = sum(wmx)
                if(max(wmx) > 0):
                    wmx[0] = wmx[0]/zmx
                    wmx[1] = wmx[1]/zmx
                    df0.loc[n, "MXSPD"] = round(sum(np.multiply(MXSPD,wmx)), 2)
                else:
                    df0.loc[n, "MXSPD"] = np.nan
                #MAX
                MAX = [float(df0.loc[n, "MAX"]), float(df1.loc[index1, "MAX"])]
                wtm = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for tmax in range(2):
                    if(MAX[tmax] > 9999):
                        MAX[tmax] = 0
                        wtm[tmax] = 0
                ztm = sum(wtm)
                if(max(wtm) > 0):
                    wtm[0] = wtm[0]/ztm
                    wtm[1] = wtm[1]/ztm
                    df0.loc[n, "MAX"] = round(sum(np.multiply(MAX,wtm)), 2)
                else:
                    df0.loc[n, "MAX"] = np.nan
                #MIN
                MIN = [float(df0.loc[n, "MIN"]), float(df1.loc[index1, "MIN"])]
                wtmi = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for tmin in range(2):
                    if(MIN[tmin] > 9999):
                        MIN[tmin] = 0
                        wtmi[tmin] = 0
                ztmi = sum(wtmi)
                if(max(wtmi)>0):
                    wtmi[0] = wtmi[0]/ztmi
                    wtmi[1] = wtmi[1]/ztmi
                    df0.loc[n, "MIN"] = round(sum(np.multiply(MIN,wtmi)), 2)
                else:
                    df0.loc[n, "MIN"] = np.nan
                #PRCP
                PRCP = [float(df0.loc[n, "PRCP"]), float(df1.loc[index1, "PRCP"])]
                wp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for temp in range(2):
                    if(PRCP[temp] > 99):
                        PRCP[temp] = 0
                        wp[temp] = 0
                zp = sum(wp)
                if(max(wp) > 0):
                    wp[0] = wp[0]/zp
                    wp[1] = wp[1]/zp
                    df0.loc[n, "PRCP"] = round(sum(np.multiply(PRCP,wp)), 2)
                else:
                    df0.loc[n, "PRCP"] = np.nan
                #SNDP
                SNDP = [float(df0.loc[n, "SNDP"]), float(df1.loc[index1, "SNDP"])]
                wsndp = [float(noro_null.loc[i, "W0"]), float(noro_null.loc[i, "W1"])]
                for sn in range(2):
                    if(SNDP[sn] > 999):
                        SNDP[sn] = 0
                        wsndp[sn] = 0
                zsndp = sum(wsndp)
                if(max(wsndp) > 0):
                    wsndp[0] = wsndp[0]/zsndp
                    wsndp[1] = wsndp[1]/zsndp
                    df0.loc[n, "SNDP"] = round(sum(np.multiply(SNDP,wsndp)), 1)
                else:
                    df0.loc[n, "SNDP"] = np.nan

                #FR
                #对FRSHTT进行修改
                fr = [df0.loc[n, "FRSHTT"], df1.loc[index1, "FRSHTT"]]
                if(((fr[0]%10) + (fr[1]%10)) > 1):# 个位
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(1)
                else:
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(0)
                if(((fr[0]//10%10) + (fr[1]//10%10)) > 1):#十位
                    df0.loc[n, "Thunder"] = int(1)
                else:
                    df0.loc[n, "Thunder"] = int(0)
                if(((fr[0]//100%10) + (fr[1]//100%10)) > 1):#百位
                    df0.loc[n, "Hail"] = int(1)
                else:
                    df0.loc[n, "Hail"] = int(0)
                if(((fr[0]//1000%10) + (fr[1]//1000%10))  > 1):#千位
                    df0.loc[n, "Snow or Ice Pellets"] = int(1)
                else:
                    df0.loc[n, "Snow or Ice Pellets"] = int(0)
                if(((fr[0]//10000 % 10) + (fr[1]//10000 % 10)) > 1):
                    df0.loc[n, "Rain or Drizzle"] = int(1)
                else:
                    df0.loc[n, "Rain or Drizzle"] = int(0)
                if(((fr[0]//100000 % 10) + (fr[1]//100000 % 10)) > 1):
                    df0.loc[n, "Fog"] = int(1)
                else:
                    df0.loc[n, "Fog"] = int(0)

            new_usaf = noro_null.loc[i, "USAF0"]
            new_wban = pd.to_numeric(noro_null.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            df0.to_csv(store_path + new_filename)


0.0000%
0.3413%
0.6826%
1.0239%
1.3652%
1.7065%
2.0478%
2.3891%
2.7304%
3.0717%
3.4130%
3.7543%
4.0956%
4.4369%
4.7782%
5.1195%
5.4608%
5.8020%
6.1433%
6.4846%
6.8259%
7.1672%
7.5085%
7.8498%
8.1911%
8.5324%
8.8737%
9.2150%
9.5563%
9.8976%
10.2389%
10.5802%
10.9215%
11.2628%
11.6041%
11.9454%
12.2867%
12.6280%
12.9693%
13.3106%
13.6519%
13.9932%
14.3345%
14.6758%
15.0171%
15.3584%
15.6997%
16.0410%
16.3823%
16.7235%
17.0648%
17.4061%
17.7474%
18.0887%
18.4300%
18.7713%
19.1126%
19.4539%
19.7952%
20.1365%
20.4778%
20.8191%
21.1604%
21.5017%
21.8430%
22.1843%
22.5256%
22.8669%
23.2082%
23.5495%
23.8908%
24.2321%
24.5734%
24.9147%
25.2560%
25.5973%
25.9386%
26.2799%
26.6212%
26.9625%
27.3038%
27.6451%
27.9863%
28.3276%
28.6689%
29.0102%
29.3515%
29.6928%
30.0341%
30.3754%
30.7167%
31.0580%
31.3993%
31.7406%
32.0819%
32.4232%
32.7645%
33.1058%
33.4471%
33.7884%
34.1297%
34.4710%
34.8123%
35.1536%
35.4949%
35.8362%
36.1775%
36.5188%
36.8601%
37.2014%
37.5427%
37.8840%
38.2253%
38.5666%
38.9

In [7]:
path = '/Users/evelyn/Desktop/weather information/noro-weather/'

path1 = '/Users/evelyn/Desktop/NORO/'

import os
import os.path
import numpy as np
import datetime

for i in range(len(noro)):
    print('{:.4%}'.format(i/len(noro)))
    
    df_outbreak = noro_station[noro_station["CDCID"] == noro.loc[i,"CDCID"]]
    df_outbreak = df_outbreak.reset_index(drop = True)

    df_null = noro_null[noro_null["CDCID"] == noro.loc[i,"CDCID"]]
    df_null = df_null.reset_index(drop = True)

    TEMP = [] ; MAX = [] ; MIN = [] ; DEWP = [] ; SLP = [] ; STP = [] ; WDSP = [] ; MXSPD = [];
    PRCP = [] ; SNDP = [];
    Fog = [] ; Rain = [] ; Snow = [] ; Hail = [] ; Thunder = [] ; Tornado = [];
    if (len(df_outbreak) > 0):
        startdate = datetime.datetime.strptime(df_outbreak.loc[0, "InitialExposure"], "%Y-%m-%d")
        enddate = datetime.datetime.strptime(df_outbreak.loc[0, "LastExposure"], "%Y-%m-%d")
    elif (len(df_null) > 0):
        startdate = datetime.datetime.strptime(df_null.loc[0, "InitialExposure"], "%Y-%m-%d")
        enddate = datetime.datetime.strptime(df_null.loc[0, "LastExposure"], "%Y-%m-%d")
    else:
        continue

    delta = datetime.timedelta(days=1)
    d = startdate

    while d <= enddate:
        yr = int(str(d)[:4])
        TEMP1 = [] ; MAX1 = [] ; MIN1 = [] ; DEWP1 = [] ; SLP1 = [] ; STP1 = [] ; WDSP1 = [] ; MXSPD1 = [];
        PRCP1 = [] ; SNDP1 = [];
        Fog1 = [] ; Rain1 = [] ; Snow1 = [] ; Hail1 = [] ; Thunder1 = [] ; Tornado1 = [];

        if (len(df_outbreak) > 0):
            for j in range(len(df_outbreak)):
                usaf = df_outbreak.loc[j, "USAF"]
                wban = df_outbreak.loc[j, "WBAN"]
                filename = str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv"
                if not(os.path.isfile(path+filename)):
                    continue 
                elif(sum(1 for line in open(path+filename)) < 10):
                    continue 
                else:
                    df = pd.read_csv(path + filename)
                    if(df.DATE.str.contains(str(d)[:10]).any()):
                        m = df[df.DATE == str(d)[:10]].index.tolist()[0]
                        if(df.loc[m, "TEMP"] < 9999):
                            TEMP1.append(df.loc[m, "TEMP"])
                        if(df.loc[m, "MAX"] < 9999):
                            MAX1.append(df.loc[m, "MAX"])
                        if(df.loc[m, "MIN"] < 9999):
                            MIN1.append(df.loc[m, "MIN"])
                        if(df.loc[m, "DEWP"] < 9999):
                            DEWP1.append(df.loc[m, "DEWP"])
                        if(df.loc[m, "SLP"] < 9999):
                            SLP1.append(df.loc[m, "SLP"])
                        if(df.loc[m, "STP"] < 100):
                            df.loc[m, "STP"] = df.loc[m, "STP"] + 1000
                            STP1.append(df.loc[m, "STP"])
                        elif not(df.loc[m, "STP_ATTRIBUTES"] == 0):
                            STP1.append(df.loc[m, "STP"]) 
                        if(df.loc[m, "WDSP"] < 999):
                            WDSP1.append(df.loc[m, "WDSP"])
                        if(df.loc[m, "MXSPD"] < 998):
                            MXSPD1.append(df.loc[m, "MXSPD"])
                        if(df.loc[m, "PRCP"] < 99):
                            PRCP1.append(df.loc[m, "PRCP"])

                        SNDP1.append(df.loc[m, "SNDP"]) 
                        Fog1.append(df.loc[m, "Fog"])
                        Rain1.append(df.loc[m, "Rain or Drizzle"])
                        Snow1.append(df.loc[m, "Snow or Ice Pellets"])
                        Hail1.append(df.loc[m, "Hail"])
                        Thunder1.append(df.loc[m, "Thunder"])
                        Tornado1.append(df.loc[m, "Tornado or Funnel Cloud"])
                    else:
                        continue

        if (len(df_null) > 0):
            for k in range(len(df_null)):
                usaf1 = df_null.loc[k, "USAF0"]
                wban1 = df_null.loc[k, "WBAN0"]
                filename1 = str(usaf1) + str(wban1) + "-" + str(yr) + "DAILY.csv"
                dfn = pd.read_csv(path1 + filename1)
                if(dfn.DATE.str.contains(str(d)[:10]).any()):
                    a = dfn[dfn.DATE == str(d)[:10]].index.tolist()[0]
                    if not(pd.isna(dfn.loc[a, "TEMP"])):
                        TEMP1.append(dfn.loc[a, "TEMP"]) 
                    if not(pd.isna(dfn.loc[a, "MAX"])):
                        MAX1.append(dfn.loc[a, "MAX"])
                    if not(pd.isna(dfn.loc[a, "MIN"])):
                        MIN1.append(dfn.loc[a, "MIN"])
                    if not(pd.isna(dfn.loc[a, "DEWP"])):
                        DEWP1.append(dfn.loc[a, "DEWP"])
                    if not(pd.isna(dfn.loc[a, "SLP"])):
                        SLP1.append(dfn.loc[a, "SLP"])
                    if not(pd.isna(dfn.loc[a, "STP"])):
                        STP1.append(dfn.loc[a, "STP"])
                    if not(pd.isna(dfn.loc[a, "WDSP"])):
                        WDSP1.append(dfn.loc[a, "WDSP"])
                    if not(pd.isna(dfn.loc[a, "MXSPD"])):
                        MXSPD1.append(dfn.loc[a, "MXSPD"])
                    if not(pd.isna(dfn.loc[a, "PRCP"])):
                        PRCP1.append(dfn.loc[a, "PRCP"])

                    SNDP1.append(dfn.loc[a, "SNDP"])
                    Fog1.append(dfn.loc[a, "Fog"])
                    Rain1.append(dfn.loc[a, "Rain or Drizzle"])
                    Snow1.append(dfn.loc[a, "Snow or Ice Pellets"])
                    Hail1.append(dfn.loc[a, "Hail"])
                    Thunder1.append(dfn.loc[a, "Thunder"])
                    Tornado1.append(dfn.loc[a, "Tornado or Funnel Cloud"])
                else:
                    continue

        if not(len(TEMP1) == 0):
            TEMP.append(np.mean(TEMP1))
        if not(len(MAX1) == 0):
            MAX.append(np.mean(MAX1))
        if not (len(MIN1) == 0):
            MIN.append(np.mean(MIN1))
        if not (len(DEWP1) == 0):
            DEWP.append(np.mean(DEWP1))
        if not (len(SLP1) == 0):
            SLP.append(np.mean(SLP1))
        if not (len(STP1) == 0):
            STP.append(np.mean(STP1))
        if not (len(WDSP1) == 0):
            WDSP.append(np.mean(WDSP1))
        if not (len(MXSPD1) == 0):
            MXSPD.append(np.mean(MXSPD1))
        if not (len(PRCP1) == 0):
            PRCP.append(np.mean(PRCP1))
        if not (len(SNDP1) == 0):
            SNDP.append(np.mean(SNDP1))
        if not (len(Fog1) == 0):
            Fog.append(np.mean(Fog1))
        if not (len(Rain1) == 0):
            Rain.append(np.mean(Rain1))
        if not (len(Snow1) == 0):
            Snow.append(np.mean(Snow1))
        if not (len(Hail1) == 0):
            Hail.append(np.mean(Hail1))
        if not (len(Thunder1) == 0):
            Thunder.append(np.mean(Thunder1))
        if not (len(Tornado1) == 0):
            Tornado.append(np.mean(Tornado1))

        d = d + delta

    noro.loc[i, "TEMP"] = np.mean(TEMP); 
    noro.loc[i, "MAX"] = np.mean(MAX); 
    noro.loc[i, "MIN"] = np.mean(MIN);
    noro.loc[i, "DEWP"] = np.mean(DEWP); 
    noro.loc[i, "SLP"] = np.mean(SLP); 
    noro.loc[i, "STP"] = np.mean(STP);
    noro.loc[i, "WDSP"] = np.mean(WDSP); 
    noro.loc[i, "MXSPD"] = np.mean(MXSPD); 
    noro.loc[i, "PRCP"] = np.mean(PRCP);
    noro.loc[i, "SNDP"] = np.mean(SNDP); 
    noro.loc[i, "Fog"] = np.mean(Fog); 
    noro.loc[i, "Rain or Drizzle"] = np.mean(Rain);
    noro.loc[i, "Snow or Ice Pellets"] = np.mean(Snow); 
    noro.loc[i, "Hail"] = np.mean(Hail); 
    noro.loc[i, "Thunder"] = np.mean(Thunder); 
    noro.loc[i, "Tornado or Funnel Cloud"] = np.mean(Tornado);  


0.0000%
0.0401%
0.0803%
0.1204%
0.1605%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid va

0.2006%
0.2408%
0.2809%
0.3210%
0.3612%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid va

0.4013%
0.4414%
0.4815%
0.5217%
0.5618%
0.6019%
0.6421%
0.6822%


...

97.9936%
98.0337%
98.0738%
98.1140%
98.1541%
98.1942%
98.2343%
98.2745%
98.3146%
98.3547%
98.3949%
98.4350%
98.4751%
98.5152%
98.5554%
98.5955%
98.6356%
98.6758%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid va

98.7159%
98.7560%
98.7961%
98.8363%
98.8764%
98.9165%
98.9567%
98.9968%
99.0369%
99.0770%
99.1172%
99.1573%
99.1974%
99.2376%
99.2777%
99.3178%


/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/evelyn/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


99.3579%
99.3981%
99.4382%
99.4783%
99.5185%
99.5586%
99.5987%
99.6388%
99.6790%
99.7191%
99.7592%
99.7994%
99.8395%
99.8796%
99.9197%
99.9599%


In [9]:
noro.to_csv('/Users/evelyn/Desktop/weather information/NORO.csv')

In [8]:
noro.describe()

,Unnamed: 0,CDCID,TotalCases,TEMP,MAX,MIN,DEWP,SLP,STP,WDSP,MXSPD,PRCP,SNDP,Fog,Rain or Drizzle,Snow or Ice Pellets,Hail,Thunder,Tornado or Funnel Cloud
count,2492.000000,2492.000000,2492.000000,2492.000000,2492.000000,2492.000000,2461.000000,2155.000000,2387.000000,2485.000000,2484.000000,2446.000000,2456.000000,2492.000000,2492.000000,2492.000000,2492.000000,2492.000000,2492.000000
mean,1245.500000,145908.865570,24.518459,49.622623,60.515222,40.244192,36.701552,1017.317343,983.891829,6.378119,13.056115,0.066716,0.160261,0.075960,0.245209,0.090772,0.000234,0.033782,0.000021
std,719.522759,133870.622318,38.425141,17.908707,19.245339,17.304621,17.450565,6.371409,44.250200,2.958132,4.130532,0.168945,0.850693,0.202335,0.346608,0.239723,0.005533,0.141484,0.001002
min,0.000000,19.000000,5.000000,-10.133333,-0.400000,-26.700000,-19.050000,992.050000,755.700000,0.150000,2.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,622.750000,10614.750000,8.000000,37.066667,46.900000,28.656250,24.600000,1013.240000,979.370312,4.295000,10.136458,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1245.500000,230579.500000,14.000000,50.417917,62.087679,40.986389,36.750000,1017.050000,993.233333,6.020000,12.561707,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1868.250000,277189.750000,26.250000,63.069444,75.844167,52.550000,49.250000,1021.265278,1009.000000,7.950000,15.368125,0.050000,0.000000,0.000000,0.444444,0.000000,0.000000,0.000000,0.000000
max,2491.000000,302706.000000,756.000000,96.200000,111.100000,81.978889,80.650000,1046.600000,1034.200000,22.800000,33.000000,2.070000,11.266667,1.000000,1.000000,1.000000,0.200000,1.000000,0.050000
